In [1]:
import warnings
warnings.filterwarnings("ignore") 

In [2]:
import pandas as pd 
df=pd.read_csv('C:\\workplace\\power-consumption\\dataset\\processed.csv',index_col=0)
df.head()

,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption
Datetime,,,,,,
2017-01-01 00:00,6.196833,75.066667,0.081833,0.063500,0.098833,22158.923532
2017-01-01 01:00,5.548833,77.583333,0.082000,0.056833,0.112500,19259.508737
2017-01-01 02:00,5.054333,78.933333,0.082333,0.063000,0.129167,17363.293880
2017-01-01 03:00,5.004333,77.083333,0.082833,0.059833,0.141000,16304.815849
2017-01-01 04:00,5.097667,74.050000,0.082333,0.058000,0.122833,15904.879449


In [3]:
df.shape

(8736, 6)

In [4]:
train_df=df.iloc[:5000-7]
y_test=df.iloc[5000-7:,-1]
test_df=df.iloc[5000-7:]

In [5]:
train_df

,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption
Datetime,,,,,,
2017-01-01 00:00,6.196833,75.066667,0.081833,0.063500,0.098833,22158.923532
2017-01-01 01:00,5.548833,77.583333,0.082000,0.056833,0.112500,19259.508737
2017-01-01 02:00,5.054333,78.933333,0.082333,0.063000,0.129167,17363.293880
2017-01-01 03:00,5.004333,77.083333,0.082833,0.059833,0.141000,16304.815849
2017-01-01 04:00,5.097667,74.050000,0.082333,0.058000,0.122833,15904.879449
...,...,...,...,...,...,...
2017-07-27 20:00,27.843333,61.208333,4.906667,13.574167,11.094667,44231.353009
2017-07-27 21:00,27.210000,62.086667,4.907500,0.083333,0.085333,42667.595168
2017-07-27 22:00,26.566667,71.410000,4.906333,0.089333,0.101167,40407.914353


In [6]:
triggered=pd.to_datetime(test_df.reset_index()['Datetime']).dt.strftime('%Y-%m-%d 00:00').unique()
len(triggered)

156

In [7]:
from sklearn.preprocessing import PowerTransformer 
scaler= PowerTransformer().fit(train_df)
train_df2=pd.DataFrame(scaler.transform(train_df),columns=train_df.columns)

In [8]:
train_df2

,Temperature,Humidity,WindSpeed,GeneralDiffuseFlows,DiffuseFlows,PowerConsumption
0,-2.519122,0.365458,-0.675575,-1.047975,-1.066473,-0.196897
1,-2.730180,0.559973,-0.675066,-1.050480,-1.060742,-0.784657
2,-2.900476,0.667014,-0.674049,-1.048163,-1.053851,-1.216934
3,-2.918187,0.520806,-0.672525,-1.049351,-1.049022,-1.478381
4,-2.885203,0.288735,-0.674049,-1.050040,-1.056457,-1.581429
...,...,...,...,...,...,...
4988,1.565400,-0.588661,1.466105,-0.053800,-0.006063,2.749115
4989,1.482930,-0.534053,1.466148,-1.040620,-1.072207,2.593667
4990,1.397928,0.094489,1.466088,-1.038422,-1.065489,2.359095
4991,1.269812,0.707137,1.465983,-1.036358,-1.065419,1.963068


In [9]:
import numpy as np
def make_supervise(train_df2,target_name,num_lags,num_steps):
    X_train=pd.DataFrame()
    for i in range(num_lags):
        for feat in train_df2.columns:
            X_train[feat+' '+'lag_{}'.format(num_lags-1-i)]=train_df2[feat].shift(num_lags-1-i)

    y_train=pd.DataFrame() 
    for i in range(1,num_steps+1):
        y_train[target_name+' '+'steps_{}'.format(i)]=train_df2[target_name].shift(-i)

    train_df3=pd.concat([X_train,y_train],axis=1).dropna()
    return train_df3

In [10]:
train_df3=make_supervise(train_df2,'PowerConsumption',24,24)

In [24]:
train_df.shape

(4993, 6)

In [27]:
144/6

24.0

In [28]:
X_train=np.array(train_df3[train_df3.columns[train_df3.columns.str.contains('lag')]]).reshape(train_df3.shape[0],24,train_df.shape[1])
y_train=np.array(train_df3[train_df3.columns[train_df3.columns.str.contains('steps')]])

In [30]:
X_train.shape

(4946, 24, 6)

In [20]:
X_train.shape,y_train.shape

((4946, 144), (4946, 24))

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,LSTM
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.callbacks import EarlyStopping

In [52]:
from sklearn.model_selection import  train_test_split 
from sklearn.metrics import mean_absolute_percentage_error
X_fit,X_val,y_fit,y_val=train_test_split(X_train,y_train,shuffle=False,test_size=0.3)

In [53]:
stop_rul=EarlyStopping(patience=10)

In [54]:
def val_fun(X_fit,y_fit,X_val,y_val,num_layers,num_nerons,alpha,iter,b_s):

    # Input layer
    model=Sequential()
    #model.add(Input(X_fit.shape[0],X_fit.shape[1],X_fit.shape[2]))
    # Hidden Layers
    for i in range(num_layers):
        if i==0:
            if num_layers>1:
                model.add(LSTM(units=num_nerons,return_sequences=True,input_shape=(X_fit.shape[1],X_fit.shape[2])))
            else:
                model.add(LSTM(units=num_nerons,return_sequences=False,input_shape=(X_fit.shape[1],X_fit.shape[2]))) 
        elif i<num_layers-1:
            model.add(LSTM(units=num_nerons,return_sequences=True))
        else :
            model.add(LSTM(units=num_nerons))
    # output layer
    model.add(Dense(units=24))
    # Training
    model.compile(optimizer=Adam(learning_rate=alpha),loss=MeanAbsoluteError())
    model.fit(X_fit,y_fit,validation_data=(X_val,y_val),epochs=iter,batch_size=b_s,callbacks=[stop_rul])
    # Evaluation of the model
    pred=model.predict(X_val)
    y_val2=scaler.inverse_transform(np.array(y_val).reshape(-1,1))
    pred2=scaler.inverse_transform(pred.reshape(-1,1))
    error=mean_absolute_percentage_error(y_val2,pred2)
    return error